In [1]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
prompt = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 개발자입니다.") , 
     ("user", "{input}")]
)
print(prompt)

prompt_text = prompt.format(input="JavaScript는 무엇인가요? 자세하게 설명해주세요.")

input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 개발자입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


In [3]:
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x000001F3C312DB80> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001F3C3536D20> root_client=<openai.OpenAI object at 0x000001F3C1D57890> root_async_client=<openai.AsyncOpenAI object at 0x000001F3C312DA90> model_name='meta-llama/llama-4-scout-17b-16e-instruct' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


In [4]:
try:
    response = llm.invoke(prompt_text)
    print(type(response))
    print(response)
    print("응답:", response.content)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'langchain_core.messages.ai.AIMessage'>
content='JavaScript는 웹 개발에서 많이 사용되는 프로그래밍 언어입니다. 이 언어는 웹 페이지에 대화형 기능을 추가하고 동적 사용자 경험을 제공하기 위해 설계되었습니다. JavaScript는 클라이언트 측 스크립팅 언어로서, 서버가 아닌 클라이언트(사용자의 웹 브라우저)에서 실행됩니다.\n\n### 역사\nJavaScript는 1995년 넷스케이프(Netscape)에서 처음 개발되었습니다. 당시에는 모카(Mocha)라는 이름으로 불렸으나, 나중에 자바스크립트(JavaScript)로 이름이 변경되었습니다. 이 언어는 자바(Java)와는 별개의 언어이며, 이름의 유사성에도 불구하고 직접적인 관련은 없습니다.\n\n### 특징\n- **동적 타이핑**: JavaScript는 동적 타이핑 언어로, 변수의 타입을 선언할 필요가 없습니다. 이는 개발의 유연성을 제공하지만, 때때로 오류를 초래할 수도 있습니다.\n- **객체 지향적**: 객체 지향 프로그래밍(OOP) 개념을 지원합니다. 객체, 상속, 다형성 등의 기능을 포함합니다.\n- **함수형 프로그래밍**: 변수에 함수를 할당할 수 있고, 함수를 인자로 전달하거나 반환받을 수 있습니다. 이는 함수형 프로그래밍의 특징입니다.\n- **클라이언트 측 실행**: 주로 웹 브라우저에서 클라이언트 측으로 실행되며, 서버로 전송되는 요청 없이도 사용자와의 상호작용을 처리할 수 있습니다.\n\n### 사용 분야\n- **웹 개발**: 웹사이트에 대화형 기능, 애니메이션, 실시간 업데이트 등을 추가하기 위해 사용됩니다. \n- **모바일 앱 개발**: React Native, Angular Mobile 등 프레임워크를 통해 모바일 애플리케이션 개발에도 사용됩니다.\n- **서버 측 개발**: Node.js를 통해 서버 측 개발도 가능합니다. Node.js는 JavaScript를 서버 측에서 실행할 수 있게 해주는 런타임 환경입니다.\

### LCEL (Prompt + LLM모델)

In [5]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("You are an expert in AI Expert. Answer the question. <Question>: {input}에 대해 쉽게 설명해주세요.")
prompt

PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an expert in AI Expert. Answer the question. <Question>: {input}에 대해 쉽게 설명해주세요.')

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# chain 연결 (LCEL)
chain = prompt | llm

# chain 호출
result = chain.invoke({"input": "인공지능 모델의 학습 원리"})
print(type(result))
print(result.content)

<class 'langchain_core.messages.ai.AIMessage'>
인공지능 모델의 학습 원리는 사람의 뇌가 학습하는 원리와 유사합니다. 우리의 뇌는 경험을 통해 배우고, 새로운 정보를 기존 지식과 연결하여 이해합니다. 인공지능 모델도 데이터를 통해 배우고, 그 데이터를 분석하여 패턴을 찾고, 이를 기반으로 예측이나 분류를 수행합니다.

보다 구체적으로 설명하면, 인공지능 모델의 학습 과정은 다음과 같습니다:

1. **데이터 수집**: 인공지능 모델을 학습시키기 위해서는大量的 데이터가 필요합니다. 이 데이터는 문제에 대한 답을 포함하고 있어야 합니다.

2. **데이터 전처리**: 수집된 데이터는 모델에 입력하기 전에 전처리 과정을 거칩니다. 이 과정에서는 데이터의 품질을 높이고, 노이즈를 제거하며, 데이터를 정규화합니다.

3. **모델 선택**: 적합한 인공지능 모델을 선택합니다. 모델의 종류에는 신경망, 결정 트리, 서포트 벡터 머신 등 여러 가지가 있으며, 문제의 성격에 따라 적절한 모델을 선택해야 합니다.

4. **학습**: 선택된 모델에 전처리된 데이터를 입력하여 학습을 시작합니다. 이 과정에서 모델은 데이터의 패턴을 분석하고, 이를 통해 예측이나 분류를 수행하는 방법을 배웁니다.

5. **평가**: 학습이 완료된 후, 모델의 성능을 평가합니다. 이를 위해서는 별도의 테스트 데이터를 사용하며, 모델의 예측 결과와 실제 값을 비교하여 정확도를 계산합니다.

6. **튜닝**: 모델의 성능이 만족스럽지 않은 경우, 모델의 파라미터를 조정하거나 학습 데이터를 추가하는 등의 방법을 통해 모델을 개선합니다.

예를 들어, 고양이와 개 사진을 분류하는 인공지능 모델을 만든다고 가정해 봅시다. 이를 위해 우리는大量的 고양이와 개 사진 데이터를 수집하고, 이를 통해 모델이 고양이와 개의 특징을 학습하도록 합니다. 모델은 이 데이터를 통해 고양이는 귀가 크고, 눈이 크며, 털이 보드라운 특징이 있고, 개는 귀가 작고, 꼬리가 긴 특징이 있다는 

### LLM 체인
- ( Prompt + LLM 모델 + 출력파서) - LCEL

In [ ]:
# 1. 프롬프트 생성
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template(
    "You are an expert in AI Expert. Answer the question. <Question>: {input}에 대해 쉽게 설명해주세요."
)
prompt

PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an expert in AI Expert. Answer the question. <Question>: {input}에 대해 쉽게 설명해주세요.')

In [ ]:
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

output_parser = StrOutputParser()

In [ ]:
# 2. chain 생성 (LCEL)
chain = prompt | llm | output_parser

In [ ]:
# 3. chain의 invoke 호출
result = chain.invoke({"input": "인공지능 모델의 학습 원리"})
print(type(result))
print(result)

<class 'str'>
<Answer>:
인공지능 모델의 학습원리는 데이터를 입력으로 받아 일련의 계산을 통해 예측을 수행하는 과정입니다. 모델은 입력 데이터와 정답 데이터 사이의 관계를 학습하여 새로운 데이터에 대해 예측을 할 수 있도록 학습됩니다. 학습 과정에서 모델은 오차를 최소화하기 위해 가중치를 조정하며, 이를 통해 데이터 간의 패턴과 관련성을 파악하게 됩니다. 이러한 학습과정을 통해 모델은 새로운 데이터에 대한 예측을 보다 정확하게 수행할 수 있게됩니다.


In [ ]:
try:
    result = chain.invoke({"input": "LangChain의 Products(제품)는 어떤 것들이 있나요?"})
    print(type(result))
    print(result)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'str'>
LangChain은 다양한 AI 기술과 서비스를 제공하는 기업입니다. LangChain의 제품에는 다국어 번역 및 통역 솔루션, 자연어 처리 기술을 활용한 챗봇 및 가상 비서 서비스, 언어 학습 및 교육 플랫폼 등이 있습니다. 또한 LangChain은 고객의 요구에 맞는 맞춤형 AI 솔루션을 개발하여 제공하고 있습니다.


### Runnable의 stream() 함수 호출

In [ ]:
# 스트리밍 출력을 위한 요청
try:
    answer = chain.stream({"input": "인공지능 모델의 학습 원리를 자세하게 설명해 주세요."})
    # 스트리밍 출력
    print(answer)
except Exception as e:
    print(f"Error: {e}")

for token in answer:
    # 스트림에서 받은 데이터의 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
    print(token, end="", flush=True)

<generator object RunnableSequence.stream at 0x000001F30CB5F2E0>
인공지능 모델의 학습 원리를 쉽게 설명해 드리겠습니다.

인공지능 모델은 데이터를 입력받아 학습을 하고, 그 데이터에서 패턴을 찾아내어 새로운 데이터를 예측하거나 분류하는 작업을 수행합니다. 이 과정은 크게 입력 데이터를 받아들이고, 이를 가공하여 출력하는 과정으로 나눌 수 있습니다.

학습 과정은 입력 데이터를 모델에 주입하고, 모델이 주어진 데이터를 분석하여 일정한 패턴이나 특징을 학습합니다. 이때 모델은 사전에 정의된 목표에 맞게 최적화되는데, 이것을 어떤 기준 함수를 사용하여 측정할 수 있습니다.

모델이 학습을 통해 데이터를 분류하거나 예측할 수 있는 이유는 이렇게 학습된 패턴을 기반으로 새로운 데이터에 대한 예측을 수행할 수 있기 때문입니다.따라서, 인공지능 모델의 핵심은 데이터를 효과적으로 분석하여 패턴을 찾아내는 것이라고 할 수 있습니다.

### Multi Chain
* 첫 번째 Chain의 출력이, 두 번째 Chain의 입력이 된다.
* 두 개의 Chain과 Prompt + OutputParser를 LCEL로 연결하기

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Step 1: 사용자가 입력한 장르에 따라 영화 추천
prompt1 = ChatPromptTemplate.from_template("{genre} 장르에서 추천할 만한 영화를 한 편 알려주세요.")

# Step 2: 추천된 영화의 줄거리를 요약
prompt2 = ChatPromptTemplate.from_template("{movie} 추전한 영화의 제목을 먼저 알려주시고, 줄을 바꾸어서 영화의 줄거리를 7문장으로 요약해 주세요.")

# OpenAI 모델 사용
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 체인 1: 영화 추천 (입력: 장르 → 출력: 영화 제목)
chain1 = prompt1 | llm | StrOutputParser()

In [ ]:
# 체인 2: 줄거리 요약 (입력: 영화 제목 → 출력: 줄거리)
chain2 = (
    {"movie": chain1}  # chain1의 출력을 movie 변수로 전달
    | prompt2
    | llm
    | StrOutputParser()
)

# 실행: "SF" 장르의 영화 추천 및 줄거리 요약
response = chain2.invoke({"genre": "액션"})
print(response)  

'매드 맥스: 분노의 도로'

- 줄거리 요약 -

맥스는 포스트 아포칼립스 세계에서 자신의 차를 빼앗긴 악당들과의 전투 속에서 미친 듯이 달리며 쫓기게 된다. 그리고 그는 무모한 여성 전사인 퍼리오사와 만나 함께 악당들을 상대로 사상 최악의 전투를 벌이게 된다. 맥스는 물자가 부족한 세상에서 살아가기 위해 불타는 소목에 직면하게 되지만, 그는 자신의 용기와 전투력으로 어려움을 극복해 나가며 포스트 아포칼립스 세계에서 살아남기 위한 모험을 떠난다. 퍼리오사와 함께한 맥스는 악당들과의 치열한 싸움 속에서 삶과 죽음을 건 결전을 펼치게 되는데, 그 과정에서 맥스는 내면의 분노와 욕망, 그리고 소망과 희망을 마주하게 되는데...


### PromptTemplate 여러개 연결하기

In [ ]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(model_name="ChatGPT", count=3)

# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요")
              + "\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요."
)
combined_prompt.format(model_name="ChatGPT", count=3, language="영어")

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
chain = combined_prompt | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3, "language":"영어"})


### 여러개의 PromptTemplate 만들기

In [ ]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

questions = [
    {"model_name": "GPT-4", "count": 2},
    {"model_name": "Gemma", "count": 3},
]

# 여러 개의 프롬프트를 미리 생성
formatted_prompts = [prompt_template.format(**q) for q in questions]
print(formatted_prompts)  # 미리 생성된 질문 목록 확인

['GPT-4 모델의 학습 원리를 2 문장으로 한국어로 답변해 주세요.', 'Gemma 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.']


In [ ]:
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

for prompt in formatted_prompts:
    response = llm.invoke(prompt)
    print(response.content)

content='GPT-4 모델은 대규모의 데이터셋을 사용하여 자기 지도 학습 방식으로 사전 훈련되며, 텍스트의 다음 단어를 예측하는 과정을 통해 문맥을 이해하고 생성합니다. 이 모델은 엄청난 양의 매개변수와 계산 능력을 가지고 있어 자연어 처리 작업에서 뛰어난 성능을 발휘합니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 132, 'prompt_tokens': 40, 'total_tokens': 172, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BgnWMYdOyniIK1zt4Q7RMrjJXUZCH', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--3df5e478-3a58-4cdf-8a18-4a4e88ae1a13-0' usage_metadata={'input_tokens': 40, 'output_tokens': 132, 'total_tokens': 172, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
content='Gemma 모델은 전이 학습을 통해 기존의 모델을 사용하여 새로운 데이터셋에 대한 학습 성능을 향상시킵니다. Gemma 모델

### ChatPromptTemplate
- SystemMessagePromptTemplate, HumanMessagePromptTemplate, 
AIMessagePromptTemplate 

In [ ]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate
)
from langchain_openai import ChatOpenAI

# 개별 메시지 템플릿 정의
system_message = SystemMessagePromptTemplate.from_template(
    "You are an {topic} expert. Please provide clear and detailed explanations."
)
user_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)
ai_message = AIMessagePromptTemplate.from_template(
    "This is an example answer about {topic}."
)

# ChatPromptTemplate로 메시지들을 묶기
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    user_message,
    ai_message
])

# 메시지 생성
messages = chat_prompt.format_messages(topic="AI", question="What is deep learning?")

# LLM 호출
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
response = llm.invoke(messages)

# 결과 출력
print(response.content)

Deep learning is a subfield of machine learning that involves building and training artificial neural networks with multiple layers (hence the term "deep"). These neural networks are designed to automatically learn and represent complex patterns and relationships within data. Deep learning has seen significant advancements in recent years, particularly with the use of deep neural networks known as convolutional neural networks (CNNs) for image recognition, recurrent neural networks (RNNs) for sequential data processing, and transformers for natural language processing tasks. Deep learning has been applied successfully in various domains such as computer vision, speech recognition, natural language processing, and reinforcement learning. Its ability to automatically learn hierarchical representations from data has led to improved performance in tasks that were previously considered challenging for traditional machine learning algorithms.


### FewShotPromptTemplate

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "input": "뉴턴의 운동 법칙을 요약해 주세요.",
        "output": """### 뉴턴의 운동 법칙
        1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.
        2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.
        3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다."""
    },
    {
        "input": "지구의 대기 구성 요소를 알려주세요.",
        "output": """### 지구 대기의 구성
        - **질소 (78%)**: 대기의 대부분을 차지합니다.
        - **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.
        - **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.
        - **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다."""
    }
]

# 예제 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 초등학생도 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델 생성 및 체인 구성
model = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
chain = final_prompt | model

# 테스트 실행
result = chain.invoke({"input": "태양계의 행성들을 간략히 정리해 주세요."})
#result = chain.invoke({"input": "양자 얽힘이 무엇인가요?"})
print(result.content)

### 태양계의 행성들

1. **수성**: 태양에 가장 가까운 행성으로, 매우 뜨겁고 낮에는 온도가 높고 밤에는 매우 차가워요.

2. **금성**: 지구와 비슷한 크기지만, 두꺼운 구름으로 덮여 있어 매우 뜨겁고, 온실 효과가 강해요.

3. **지구**: 생명체가 살 수 있는 유일한 행성으로, 물이 액체 상태로 존재해요.

4. **화성**: 붉은색을 띠고 있으며, 물이 있었던 흔적이 있어요. 탐사 로봇들이 많이 가고 있어요.

5. **목성**: 태양계에서 가장 큰 행성으로, 많은 위성을 가지고 있고, 대적반이라는 큰 폭풍이 있어요.

6. **토성**: 아름다운 고리로 유명한 행성으로, 가스 행성이에요.

7. **천왕성**: 옆으로 누워서 돌고 있는 독특한 행성으로, 푸른색을 띠고 있어요.

8. **해왕성**: 태양계에서 가장 먼 행성으로, 강한 바람과 폭풍이 특징이에요. 

이렇게 태양계에는 총 8개의 행성이 있어요!


### PartialPromptTemplate
* 프롬프트의 입력값을 동적으로 활용

In [12]:
from datetime import datetime
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month
    
    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# 프롬프트 템플릿 정의 (부분 변수 적용)
# prompt = PromptTemplate(
#     template="{season}에 일어나는 대표적인 지구과학 현상은 {phenomenon}입니다.",
#     input_variables=["phenomenon"],  # 사용자 입력 필요
#     partial_variables={"season": get_current_season()}  # 동적으로 계절 값 할당
# )


season = get_current_season("south")

prompt = ChatPromptTemplate.from_template(
    "{season}에 주로 발생하는 대표적인 지구과학 현상 3가지를 알려주세요. "
    "각 현상에 대해 간단한 설명을 포함해주세요."
)

parser = StrOutputParser()

# OpenAI 모델 초기화
model = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 특정 계절의 현상 질의
chain = (
    {"season": lambda x: season}
    | prompt
    | model
    | parser
)

result = chain.invoke({})

# 결과 출력
print(f"🔹 프롬프트: {prompt}")
print(f"🔹 모델 응답: {season}에 발생하는 자연 현상 : \n {result}")

🔹 프롬프트: input_variables=['season'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['season'], input_types={}, partial_variables={}, template='{season}에 주로 발생하는 대표적인 지구과학 현상 3가지를 알려주세요. 각 현상에 대해 간단한 설명을 포함해주세요.'), additional_kwargs={})]
🔹 모델 응답: 겨울에 발생하는 자연 현상 : 
 1.  **극광**: 극지방에서 발생하는 현상으로 태양풍이 지구 자기장에 의해 극지방으로 유입되어 대기 중의 원자와 분자와 충돌하면서 발생하는 빛입니다. 태양풍의 강도가 강해지면 극광이 더 밝고 자주 발생하며, 극광은 북극에서는 오로라 보레알리스(Aurora Borealis), 남극에서는 오로라 오스트랄리스(Aurora Australis)로 불립니다.
2.  **성층권의 온도가 낮아짐**: 성층권은 대기의 중간층으로, 일반적으로 고도가 높을수록 온도가 낮아지는 경향이 있습니다. 하지만 겨울에는 성층권의 온도가 더 낮아지는 현상이 발생합니다. 이로 인해 상층 대기가 극도로 차가워지고, 이는 대기 순환과 기상 현상에 영향을 줍니다.
3.  **빙상 형성**: 겨울에는 기온이 낮아지면서 지표면의 물이 얼어 빙상이 형성되는 현상이 발생합니다. 특히, 습한 토양이나 호수, 강 등에서 물이 얼어서 빙상이 형성되며, 이로 인해 생태계와 인간 활동에 영향을 줍니다.


In [10]:
import requests
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 실시간 환율을 가져오는 함수
def get_exchange_rate():
    response = requests.get("https://api.exchangerate-api.com/v4/latest/USD")
    data = response.json()
    return f"1달러 = {data['rates']['KRW']}원"

# {info} 변수에 API에서 받은 환율 정보를 동적으로 반영
prompt = PromptTemplate(
    template="현재 {info} 기준으로 환율 정보를 알려드립니다. 현재 환율을 기준으로 한국 경제에 미친 영향에 대한 분석을 제공해 주세요.",
    input_variables=[],  # 사용자 입력 없음
    partial_variables={"info": get_exchange_rate()}  # API에서 가져온 데이터 자동 반영
)

# LLM 모델 설정 (GPT-4o-mini 사용)
model = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 모델에 프롬프트 전달 및 응답 받기
response = model.invoke(prompt.format())

# 결과 출력
print("🔹 프롬프트:", prompt.format())
print("🔹 모델 응답:", response.content)

🔹 프롬프트: 현재 1달러 = 1365.14원 기준으로 환율 정보를 알려드립니다. 현재 환율을 기준으로 한국 경제에 미친 영향에 대한 분석을 제공해 주세요.
🔹 모델 응답: ### 한국 경제에 미친 영향

1. **수출 증가**: 높은 환율은 한국의 수출 산업에 긍정적인 영향을 미칩니다. 한국 제품이 해외 시장에서 더 저렴해지기 때문에, 수출량이 증가할 수 있습니다. 이는 자동차, 반도체, 철강 등 주요 수출 품목의 경쟁력을 높여줍니다.

2. **수입 비용 증가**: 반면에, 높은 환율은 수입 비용을 증가시킵니다. 원유, 원자재, 식품 등 수입에 의존하는 산업의 비용이 증가하여, 국내 물가 상승으로 이어질 수 있습니다. 이는 특히 에너지와 식품 가격에 민감한 가계와 기업에 부담을 줄 수 있습니다.

3. **물가 상승**: 수입 물가가 상승하면, 최종 소비재 가격도 상승할 가능성이 있습니다. 이는 가계의 구매력을 감소시키고, 전반적인 경제 성장에 부정적인 영향을 미칠 수 있습니다.

4. **관광객 유치**: 높은 환율은 한국을 방문하는 외국인 관광객에게 유리합니다. 관광객들은 더 많은 원화를 얻을 수 있기 때문에, 한국에서의 소비가 증가할 수 있습니다. 이는 관광 산업에 긍정적인 영향을 미칩니다.

5. **외환시장 변동성**: 높은 환율은 외환시장의 변동성을 증가시킬 수 있습니다. 이는 환율의 급격한 변동으로 인해 기업과 투자자에게 불확실성을 증가시키고, 리스크 관리에 어려움을 줄 수 있습니다.

6. **기업의 수익성**: 수출 중심의 기업들은 높은 환율로 인해 수익성이 개선될 수 있지만, 수입 의존도가 높은 기업들은 비용 증가로 인해 수익성이 감소할 수 있습니다.

### 종합 분석

현재의 높은 환율은 한국의 수출 산업에 긍정적인 영향을 미치고 있지만, 수입 비용 증가와 물가 상승이라는 부정적인 영향도 동반하고 있습니다. 따라서, 정부는 환율의 안정성을 유지하고, 경제 전반의 균형을 맞추기 위해 적절한 통화 정책을 펼쳐야 할 것입니다. 또한, 기업들